## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Rikitea,PF,-23.1203,-134.9692,78.40,69,90,4.99,overcast clouds
1,1,Nanortalik,GL,60.1432,-45.2371,7.34,67,41,15.77,scattered clouds
2,2,Ilulissat,GL,69.2167,-51.1000,3.22,51,100,6.91,overcast clouds
3,3,Pisco,PE,-13.7000,-76.2167,69.85,78,0,14.97,clear sky
4,4,De Land,US,29.0283,-81.3031,60.15,83,4,4.81,clear sky


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min = float(input("Please enter a minimum temperature: "))
max = float(input("Please enter a maximum temperature: "))

Please enter a minimum temperature: 75
Please enter a maximum temperature: 80


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
criteria_df = city_data_df.loc[(city_data_df['Max Temp']>=min) & (city_data_df['Max Temp']<=max)]
criteria_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Rikitea,PF,-23.1203,-134.9692,78.40,69,90,4.99,overcast clouds
12,12,Batticaloa,LK,7.7102,81.6924,75.90,70,4,0.78,clear sky
14,14,Victoria,SC,-4.6167,55.4500,78.06,83,40,5.75,scattered clouds
15,15,Busselton,AU,-33.6500,115.3333,77.04,36,48,12.91,scattered clouds
29,29,Mayumba,GA,-3.4320,10.6554,77.04,90,99,5.44,moderate rain
...,...,...,...,...,...,...,...,...,...,...
616,616,Weligama,LK,5.9667,80.4167,76.87,69,15,9.75,few clouds
619,619,Cururupu,BR,-1.8283,-44.8683,75.25,95,76,4.41,broken clouds
621,621,Macau,BR,-5.1150,-36.6344,79.92,77,53,13.38,broken clouds
634,634,West Bay,KY,19.3667,-81.4167,75.27,89,0,3.00,clear sky


In [7]:
# 4a. Determine if there are any empty rows.
criteria_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [8]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = criteria_df.dropna()

In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Rikitea,PF,78.40,overcast clouds,-23.1203,-134.9692,
12,Batticaloa,LK,75.90,clear sky,7.7102,81.6924,
14,Victoria,SC,78.06,scattered clouds,-4.6167,55.4500,
15,Busselton,AU,77.04,scattered clouds,-33.6500,115.3333,
29,Mayumba,GA,77.04,moderate rain,-3.4320,10.6554,
35,Arraial Do Cabo,BR,75.63,overcast clouds,-22.9661,-42.0278,
40,Humberto De Campos,BR,75.31,broken clouds,-2.5983,-43.4611,
48,Kapaa,US,79.12,clear sky,22.0752,-159.3190,
49,The Valley,AI,77.83,clear sky,18.2170,-63.0578,
66,Bubaque,GW,77.81,clear sky,11.2833,-15.8333,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat=row['Lat']
    lng=row['Lng']
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params["location"] = f"{lat},{lng}"
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [11]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[hotel_df['Hotel Name']!=""]
clean_hotel_df.head(60)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Rikitea,PF,78.40,overcast clouds,-23.1203,-134.9692,People ThankYou
12,Batticaloa,LK,75.90,clear sky,7.7102,81.6924,Grand Star Guest House
14,Victoria,SC,78.06,scattered clouds,-4.6167,55.4500,Coral Strand Smart Choice Hotel
15,Busselton,AU,77.04,scattered clouds,-33.6500,115.3333,Observatory Guest House
29,Mayumba,GA,77.04,moderate rain,-3.4320,10.6554,LIKWALE LODGE
35,Arraial Do Cabo,BR,75.63,overcast clouds,-22.9661,-42.0278,Pousada Porto Praia
40,Humberto De Campos,BR,75.31,broken clouds,-2.5983,-43.4611,Pousada do Cabeça
48,Kapaa,US,79.12,clear sky,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
49,The Valley,AI,77.83,clear sky,18.2170,-63.0578,CeBlue Villas & Beach Resort
66,Bubaque,GW,77.81,clear sky,11.2833,-15.8333,"Casa Dora, Bubaque"


In [13]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [14]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [15]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))